In [ ]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback


In [ ]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
def make_valid(tname):
    c = "UPDATE %s SET geom=ST_MakeValid(geom) WHERE ST_IsValid(geom) = false;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    
    

In [ ]:
def make_valid_gid(tname,gid):
    c = "UPDATE %s SET geom=ST_MakeValid(geom) WHERE gid = %s and ST_IsValid(geom) = false;" % (tname, gid)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    


In [ ]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name)
tnames.sort()

In [ ]:
len(tnames),
tnames[16:32]

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=12)

for tname in tnames:
    exe.submit(make_valid, tname)
exe.shutdown()
None

In [ ]:
# make_valid_range('fw_plants', 0, 25)

In [ ]:
#exe = SimpleProcessPoolExecutor(max_workers=16)

#for i in range(0,3169):
#    exe.submit(make_valid_gid, 'fw_plants', i)
#exe.shutdown()
#None